In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = (
    SparkSession.builder
    .appName("MySql integration")
    .config("spark.jars", "/home/wedivv/Downloads/mysql-connector-j-8.0.33.jar")
    .getOrCreate()
)

23/05/21 01:24:09 WARN Utils: Your hostname, wedivv-H110M-S2V resolves to a loopback address: 127.0.1.1; using 192.168.1.44 instead (on interface wlp5s0)
23/05/21 01:24:09 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
23/05/21 01:24:09 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [3]:
properties = {
    "driver": "com.mysql.cj.jdbc.Driver",
    "user": "username",
    "password": "Pass1234*"
}

In [4]:
jdbcDF = ( 
    spark
        .read
        .format("jdbc")
        .option("url", "jdbc:mysql://localhost:3306/sakila")
        .option("dbtable", "customer")
        .options(**properties)
        .load() 
        )

# The double asterisks (**) are used to unpack the properties dictionary 
# and pass its contents as individual key-value arguments

jdbcDF.show()


+-----------+--------+----------+---------+--------------------+----------+------+-------------------+-------------------+
|customer_id|store_id|first_name|last_name|               email|address_id|active|        create_date|        last_update|
+-----------+--------+----------+---------+--------------------+----------+------+-------------------+-------------------+
|          1|       1|      MARY|    SMITH|MARY.SMITH@sakila...|         5|  true|2006-02-14 22:04:36|2006-02-15 04:57:20|
|          2|       1|  PATRICIA|  JOHNSON|PATRICIA.JOHNSON@...|         6|  true|2006-02-14 22:04:36|2006-02-15 04:57:20|
|          3|       1|     LINDA| WILLIAMS|LINDA.WILLIAMS@sa...|         7|  true|2006-02-14 22:04:36|2006-02-15 04:57:20|
|          4|       2|   BARBARA|    JONES|BARBARA.JONES@sak...|         8|  true|2006-02-14 22:04:36|2006-02-15 04:57:20|
|          5|       1| ELIZABETH|    BROWN|ELIZABETH.BROWN@s...|         9|  true|2006-02-14 22:04:36|2006-02-15 04:57:20|
|          6|   

##### inserting our own data

In [5]:
from datetime import datetime

data = [
    (None, 1, "Joaozinskyvoiqueinsverson", "da Silva", 
     "joaozika_verson@ig.com.br", 1, 1, 
     datetime.now(), datetime.now())]

In [6]:
from pyspark.sql.types import *

schema = StructType([
    StructField("customer_id", IntegerType(), nullable=True),
    StructField("store_id", IntegerType(), nullable=False),
    StructField("first_name", StringType(), nullable=False),
    StructField("last_name", StringType(), nullable=False),
    StructField("email", StringType(), nullable=True),
    StructField("address_id", IntegerType(), nullable=False),
    StructField("active", IntegerType(), nullable=False),
    StructField("create_date", TimestampType(), nullable=False),
    StructField("last_update", TimestampType(), nullable=False)
])

In [7]:
df = spark.createDataFrame(data, schema)

In [8]:
df.write \
    .format("jdbc") \
    .option("url", "jdbc:mysql://localhost:3306/sakila") \
    .option("dbtable", "customer") \
    .options(**properties) \
    .mode("append") \
    .save()


###### Achando o Joaozinskyvoiquinsverson da Silva

In [9]:
jdbcDF = ( 
    spark
        .read
        .format("jdbc")
        .option("url", "jdbc:mysql://localhost:3306/sakila")
        .option("dbtable", "customer")
        .options(**properties)
        .load() 
        )


jdbcDF.where("first_name = 'Joaozinskyvoiqueinsverson'").show(truncate=False)

+-----------+--------+-------------------------+---------+-------------------------+----------+------+-------------------+-------------------+
|customer_id|store_id|first_name               |last_name|email                    |address_id|active|create_date        |last_update        |
+-----------+--------+-------------------------+---------+-------------------------+----------+------+-------------------+-------------------+
|602        |1       |Joaozinskyvoiqueinsverson|da Silva |joaozika_verson@ig.com.br|1         |true  |2023-05-21 01:24:17|2023-05-21 01:24:16|
+-----------+--------+-------------------------+---------+-------------------------+----------+------+-------------------+-------------------+



##### dbs

In [10]:
databasesDF = spark.read \
    .format("jdbc") \
    .option("url", "jdbc:mysql://localhost:3306/information_schema") \
    .option("query", "SELECT SCHEMA_NAME FROM SCHEMATA") \
    .options(**properties) \
    .load()

# Show the list of databases
databasesDF.show()

+------------------+
|       SCHEMA_NAME|
+------------------+
|             mysql|
|information_schema|
|performance_schema|
|               sys|
|           desafio|
|            sakila|
|       board_games|
|          datapipe|
+------------------+

